# Лабораторная работа №7

## **Исследование простейших нейронных сетей с использованием Scikit-Learn.**

Библиотека Scikit-Learn поддерживает работу с простейшим типом нейронных сетей на базе изученного в работе 6 персептрона, в частности имеется возможность использования класса многослойный персептрон (MLP) как для задач классификации, так и для задач регрессии.

К недостаткам многослойного персептрона (MLP) можно отнести:

- MLP со скрытыми слоями имеют невыпуклую функцию потерь, когда существует более одного локального минимума. Поэтому разные инициализации случайных весов могут привести к разной точности проверки.
- MLP требует настройки ряда гиперпараметров, таких как количество скрытых нейронов, слоев и итераций.
- MLP чувствителен к масштабированию функций.

<br/><br/>


**Задания:**

**Часть 1. Изучение многослойного персептрона для задачи регресии.**

Будем использовать класс **[MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor)** применительно к используемому ранее датасету с данными по недвижимости в Боостоне. Необходимо:

1. Реализовать модель машинного обучения с использованием [MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor). Вывести метрики модели. Сравнить полученные результаты с результатами предыдущих исследований этого датасета.
1. Исследовать зависимость качества получаемых моделейпри использовании класса [MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor) от гиперпараметров влияющих на конфигурацию нейронной сети (количество слоев, скрытых нейронов). Какой из параметров оказывает наибольшее влияние на модель и почему.


In [7]:
import pandas as pd

# Read the data from the file
boston = pd.read_csv('boston_house_prices.csv')
# set header names from first row
boston.columns = boston.iloc[0]
boston = boston.drop(0)
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
1,0.00632,18,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24
2,0.02731,0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.9,9.14,21.6
3,0.02729,0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
4,0.03237,0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.9,5.33,36.2


In [18]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tabulate import tabulate

X = boston.drop('MEDV', axis=1)
y = boston['MEDV']

# разбиение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

# инициализация и обучение модели MLPRegressor
model = MLPRegressor(hidden_layer_sizes=(100, 100),
                     max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# предсказание на тестовых данных
y_pred = model.predict(X_test)

# вычисление метрик модели
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# сохранение метрик в таблицу для сравнения (лаб2, лаб4)
ridge_mse = 2243.84
ridge_r2 = 0.284259
ridge_mae = 33.0214

lasso_mse = 2258.75
lasso_r2 = 0.279504
lasso_mae = 32.9439

linear_mse = 23.3808
linear_r2 = 0.7315
linear_mae = 3.27

tree_mse = 17.3940
tree_r2 = '-'
tree_mae = '-'

random_forest_mse = 8.7232
random_forest_r2 = '-'
random_forest_mae = '-'

# сравнение с предыдущими результатами
table = [["Ridge", ridge_mse, ridge_r2, ridge_mae],
         ["Lasso", lasso_mse, lasso_r2, lasso_mae],
         ["Linear", linear_mse, linear_r2, linear_mae],
         ["Tree", tree_mse, tree_r2, tree_mae],
         ["Random Forest", random_forest_mse, random_forest_r2, random_forest_mae],
         ["MLPRegressor", mse, r2, mae]]
print(tabulate(table, headers=[
      "Model", "MSE", "R2", "MAE"], tablefmt="github"))

| Model         |       MSE | R2                 | MAE                |
|---------------|-----------|--------------------|--------------------|
| Ridge         | 2243.84   | 0.284259           | 33.0214            |
| Lasso         | 2258.75   | 0.279504           | 32.9439            |
| Linear        |   23.3808 | 0.7315             | 3.27               |
| Tree          |   17.394  | -                  | -                  |
| Random Forest |    8.7232 | -                  | -                  |
| MLPRegressor  |   19.0245 | 0.7283272570005634 | 3.0599342993661875 |


Модель MLPRegressor показала значительно лучшие результаты по сравнению с линейными моделями. Однако она все еще не может сравниться с деревьями решений. При этом, в отличие от деревьев решений, MLPRegressor не имеет проблемы переобучения.


**Часть 2. Изучение многослойного персептрона для задачи классификации.**

В этой части будем использовать класс **[MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)** и новый для нас датасет. Попробуем обучить нашу простую нейросеть различать рукописные цифры (датасет типа MNIST). Он включает в себя изображения цифр, написанных от руки, с соответствующими ярлыками, которые объясняют, что это за число. Каждое изображение размером 8х8 пикселей. Загрузить и посмотреть на пример можно следующим образом:

![](Aspose.Words.68e21292-7d96-4035-bb10-a225de022a4a.001.png)

Наш код будет анализировать цифры, которые соответствуют пикселям изображения. Вот они –

![](Aspose.Words.68e21292-7d96-4035-bb10-a225de022a4a.002.png)

Однако перед тем, как начинать строить модель машинного обучения, нам необходимо провести предобработку данных, которая в нашем случае будет заключатся в масштабировании. В начале мы отмечали, что MLP чувствителен к масштабированию, вспомните линейные модели и вы поймете почему. Обратите внимание, что в приведенном примере - диапазон от 0 до 15. Значения отличаются на порядок, что достаточно много. Проведем масштабирование стандартными средствами библиотеки.

![](Aspose.Words.68e21292-7d96-4035-bb10-a225de022a4a.003.png)

Теперь можно можно приступать к построению модели. Обратите внимание, что мы имеем задачу классификации по 10 классам.

Необходимо.

1. Завершить построение модели многоклассовой классификации с использованием [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier).
1. Вывести точность предсказания для каждого из классов.


In [23]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Load dataset
digits = load_digits()

# Scale data
scaler = StandardScaler()
X = scaler.fit_transform(digits.data)

# Assign target values
y = digits.target

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Create MLPClassifier model
mlp = MLPClassifier(hidden_layer_sizes=(12, 12),
                    max_iter=1000, random_state=42)

# Fit the MLP classifier to the training data
mlp.fit(X_train, y_train)

# Make predictions for the test data
y_pred = mlp.predict(X_test)

# Calculate accuracy score for each class
accuracies = []
for i in range(10):
    accuracies.append(accuracy_score(y_test == i, y_pred == i))

# Print accuracy scores
print(tabulate([[i, accuracies[i]] for i in range(10)], headers=[
    "Digit", "Accuracy"], tablefmt="github"))

|   Digit |   Accuracy |
|---------|------------|
|       0 |   0.994444 |
|       1 |   0.991667 |
|       2 |   0.991667 |
|       3 |   0.997222 |
|       4 |   0.994444 |
|       5 |   0.986111 |
|       6 |   0.994444 |
|       7 |   0.991667 |
|       8 |   0.977778 |
|       9 |   0.986111 |
